In [1]:
from flask import Flask, request, jsonify
from PIL import Image
import torch
from ultralytics import YOLO
import os

app = Flask(__name__)

# Load the pre-trained YOLO model
model = YOLO('best.pt')

# Define classes
classes = ['mitosis', 'normal']

@app.route('/predict', methods=['GET'])
def predict():
    # Get the image filename from the request query parameter
    filename = request.args.get('image_filename', '')
    
    # Validate the image_filename
    if not filename:
        return jsonify({'error': 'image_filename parameter is missing.'}), 400
    
    # Read the image file
    base_path = r"storage\app\public\images"
    
    image_path = os.path.join(base_path, filename)

    image_path = image_path.replace('\\', '/')
    try:
        image = Image.open(image_path).convert('RGB')
    except Exception as e:
        return jsonify({'error': f'Error reading the image file: {image_path}'}), 400
    
    # Perform prediction
    with torch.no_grad():
        outputs = model(image)

    # Get prediction result
    prediction = classes[outputs[0].probs.data.argmax()]
    print("Predicted class:", prediction)
    
    # Return prediction result
    return jsonify({'prediction': prediction, 'image_filename': filename, 'outputs':outputs[0].probs.data.tolist()})

if __name__ == '__main__':
    try:
        app.debug = True
        app.run(host='0.0.0.0', use_reloader=False, port=5001)
    except Exception as e:
        print(f"Exception Occurred: {e}")

d:\Study\Anaconda\envs\audio_env2\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
 * Tip: There are .env or .flaskenv files present. Do "pip install python-dotenv" to use them.


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.1.3:5001
Press CTRL+C to quit
